In [1]:
from map_manager.map_generator import map
from map_manager.world_encounters import *
import pickle
from matplotlib import colors
from tools.utils import required_n_enc
from quest_engine.quest_generator import *
from map_manager.noise import toddler
import matplotlib.pyplot as plt
from tools.utils import basic_distance

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [3]:
#nw = map()

#nw.populate_map(n_locations=1024, name="terrain",relaxed=True,k=100)
#nw.populate_map(n_locations=2048, name="civilisation",relaxed=True,k=250)
#nw.populate_map(n_locations=1024, name="threat",relaxed=True, k=100)
#nw.populate_map(n_locations=64, name="story_arc",relaxed=True, k=100)
#
#
#nw.attribute_view(seed_1=24,seed_2=34,map_name="terrain",view_name="terrain")
#nw.attribute_view(seed_1=98,seed_2=50,map_name="civilisation",view_name="civilisation")
#nw.attribute_view(seed_1=34,seed_2=34,map_name="threat",view_name="threat",double=False)
#nw.attribute_view(seed_1=74,seed_2=74,map_name="story_arc",view_name="story_arc",double=True)
#
#
#nw.attribute_centroids("civilisation")
#nw.attribute_centroids("terrain")
#nw.land_mask()
#
#
#with open('map_nw', 'wb') as f:
#    # Pickle the 'data' dictionary using the highest protocol available.
#    pickle.dump(nw, f)


with open("map_nw", "rb") as f:
    nw = pickle.load(f)



#### - Questin ilk ve son adımının birden fazla keyi olabilir.(Init)
#### - Arc arttıkça, questin o arc bölgesinde bitmesi şart. (Fitness func)

In [4]:
q = quest(nw)

In [5]:
q.key_coords

[[[455.3, 501.8], [94.1, 146.1], [706.8, 828.6]],
 [[609.0, 380.3]],
 [[445.0, 193.2]],
 [[32.7, 330.4]]]

In [6]:
q.lock_coords

[[],
 [[328.1, 273.7]],
 [[1000.1, 254.6]],
 [[51.2, 60.0], [137.0, 1006.0], [932.9, 289.3]]]

In [21]:
from turtle import distance


distances = []

for i in range(q.steps-1):
    dists = [(basic_distance(coord,q.key_coords[i+1][0])) for coord in q.key_coords[i]]
    distances.append(dists)
print(distances)
    
    
                

[[195.92330131967458, 565.6603662976574, 458.8439059200852], [248.8019493492766], [434.52862966667686]]


In [22]:
q.key_coords

[[[455.3, 501.8], [94.1, 146.1], [706.8, 828.6]],
 [[609.0, 380.3]],
 [[445.0, 193.2]],
 [[32.7, 330.4]]]

In [ ]:
cmap = colors.ListedColormap([ "blue","white","yellowgreen","khaki","lawngreen","slategrey","darkgreen"])
cmap_arcs = colors.ListedColormap(["blue","khaki","khaki","black",])

ter = nw.views["terrain"].copy()
civ = nw.views["civilisation"].copy()
thr = nw.views["threat"].copy()
arc = nw.views["story_arc"].copy()


fig, ax = plt.subplots(1,4)
fig.set_dpi(150)
fig.set_size_inches(12, 8)

ax[0].imshow(ter.T,cmap=cmap,alpha=.66)
ax[0].set_title("Terrain")

ax[1].imshow(civ.T,cmap=cmap_arcs,alpha=0.66)
ax[1].set_title("Civilisation")

ax[2].imshow(thr.T,cmap="Reds",alpha=0.66)
ax[2].set_title("Threat")

ax[3].imshow(arc.T,cmap="Greens",alpha=0.66)
ax[3].set_title("Story Arc")


In [ ]:
nw.views["civilisation"]